---
title: Primitives examples
description: Practical examples of using primitives in Qiskit Runtime.
---


# Primitives examples

<Admonition type="note" title="New execution model, now in beta release">
The beta release of a new execution model is now available. The directed execution model provides more flexibility when customizing your error mitigation workflow. See the [Directed execution model](/docs/guides/directed-execution-model) guide for more information.
</Admonition>

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<Accordion>
<AccordionItem title="Package versions">

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</AccordionItem>
</Accordion>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.22341568206229862
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: -0.32345013477088946
>>> Standard errors for PUB 0: 0.6699759406700503
>>> Expectation values for PUB 1: -0.06611570247933884
>>> Standard errors for PUB 1: 0.9963717541237429
>>> Expectation values for PUB 2: -1.027027027027027
>>> Standard errors for PUB 2: 0.5749927616830978


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.06711015  1.02462229  0.86305279  0.62488953  0.33325515 -0.00896003
  -0.33007578 -0.6569722  -0.86218569 -1.00699127 -1.07260178 -1.01652936
  -0.84628887 -0.61275014 -0.34655067  0.00202323  0.3451055   0.65176961
   0.88357414  1.02202099  1.08069471]
 [ 0.02023232  0.36013522  0.67142386  0.90582969  1.03936298  1.09630249
   1.02375519  0.83848498  0.61506241  0.29134535 -0.02312265 -0.38557014
  -0.68269615 -0.88415221 -1.03242618 -1.07462501 -1.01479516 -0.84657791
  -0.60032172 -0.34134807  0.0375743 ]
 [-0.00375743 -0.32227189 -0.61043788 -0.8595844  -0.99514091 -1.06768822
  -1.00207771 -0.86016246 -0.61535144 -0.31128863 -0.00722583  0.3162022
   0.5783552   0.83472755  0.96190211  1.0543927   0.96219114  0.83183722
   0.61477337  0.3109996   0.00578066]
 [ 1.05121333  0.97808796  0.82807979  0.60841465  0.28643179  0.00202323
  -0.35955716 -0.61650757 -0.87403605 -1.01017063 -1.03907394 -0.97201827
  -0.82576753 -0.64078635 -0.29018922  0.01445

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: -1.4598540145985401
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: -0.783068783068783
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1100110110101001110011101111010011011000000011001000010010010000000000100000000100001001101001111001110011100001110000010010001', '1110011011100000011110111010110100100001000000001110110100001110100001101100010101110001100001101001000101001101000000001010010', '1110001110011101010000110011111011000001000101101000010000011110000000001101010011010011001001100000111000000000100010101011101', '1000010100001000101010010000000110111111101001001111001010101100101111010011000000110010000000110010010001000010000000011000000', '1110010001000101110101001010101111000000110010000011100001100110101000010111010001000100110010000100011101001110000001010010001', '0001110001100111100001101101010110001110000100101010011110011001100100110110110100010111100000001001010110100010001001010001000', '0001100100000010001011011110101110101100111001101011100110111100100000010100010101101000000001000010110011001010101000000000000', '00100100100101001001001100101000010000000000000101100101101

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0101110010001111101011000010000011000011100101001000111001000110101001100000000100000110010001000010110100111111001000101111001', '0110001110000000011110001110011001011111001101011100101010100110100111000010101110010100100010000000101101110100001110000000000', '0010100100010011000010000011101110101100010001001100011000000011011011101110101001111110001001000000100000001000100010001111110', '0011010011110111100100011010000111000010111001000000000000101100100100000000111101101100111110010101001000100000101000000101011', '0100100101010111010011110111111100101011011101000110100100110000000011111000001000011110000110010100000100000111010000000000100', '1011000101011010000000010110111111000001011010101101011001111001010110010100000100001100010000010000110100000010010000110000010', '0110110101001000010010111011100100001100110010111001100011101100011000010100010000011111001110100011011110000111010000100000000', '0010010111010000110101001101110111010100001010001

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['1110001001011111101111010101101110101110000101100010101100001101101100001000000010111111111001100100111001100000001000101011101', '0001001110111001000010010110110111110111100000100011000111011100110011001000001011011110011011000111110001110010100001000000000', '0000100100110011000000011000010001101010100000100011111110101111010110111101100110100000011000110100001000110001001101000010111', '0010000000010111101001010011101000110000110000100000010000111011010101001001101110111010110011100010111010011100111010111011111', '0001100101010111000000110111110010000110101101100011000001000011010100011001101101001001000000100110110100100010110010101100001', '0000111111001001011000100101100100111000000001111001001110010011010000010011101000011000010011110110110101111011000101000000101', '1111100000101000010000000011100100100110100101011111010111101010010000001011100110101111100100111000001001100010011110000000100', '11000110001001100000100110001

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1000001010111001110000111111100010001010101001001000010000000101101110111100110111001110010010111010010001001110010110000000000', '1010000100010101101000110100100111101110100101010010110000110000010110011000110111010110000001010111100000000000001000001101000', '1010101111110101010001111101000010011001111001110000110010000100000010000000000101111110101011100000000001000110101010110001100', '0001000100000010001011111101110011000010011001100100101101100011000111100010010100010110000000010011010000000011101010000100000', '0010110111011011001110000000101000000100010101010100010000101101000011010011001101000110110110001000100000111110011101100111000', '0001111111001110001010001000110011000001001100101110100000000011110110000000101000010011000001101010111100000001000101000001101', '1010111111000000001000010001111111100100110000010010101000001110111011000010101010010100000000110100110110000110100100101000010', '1111010110011100001100000001100010

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1100111001001110001001001010001000111000100001010000000011010100101100001100001101010100010001000001011101110000000000010001000', '1100001010011111100101011000100000010010011100110100011011000110000011010010010100010000000111011001000001001001000000010011100', '1010101001101100101101111110000011101010000100001001001111000100001001100100000100011001000101100010101000100100101100000100001', '1110110010101001110000111100001010011101111110001000000111000001011011000011001001001011001000111100011000001111100100100101001', '0101011001001101111100010100000000010001010000110110010010111000010100001001101100000101010000100100110111011001010100111100110', '0000111000100110001100111010111011011000000010001011011111010000010110100011100100110101011100001001011010000001000001010010100', '1111100000001110111100101011001110100010001110000101010111011010100011110100110000000100011000001010101110100000110000100100001', '0011001000101001101011000110100110

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Compare transpiler settings](/docs/guides/circuit-transpilation-settings) guide.
    - Read [Migrate to V2 primitives](/docs/guides/v2-primitives).
    - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>